In [1]:
import numpy as np
from sklearn import datasets,linear_model
import matplotlib.pyplot as plt

%matplotlib inline


class Evaluation_Metric:
    
    def __init__(self,account):
        self.account = account
        
#   年均回报率
    def year_avg_return_rate(self):
        asset_df = self.account.get_asset_df()
        
        last_day_asset = asset_df[asset_df.shape[0]-1:]        
        invest_return = last_day_asset['assets'].values[0]
        
        avg_rate = (invest_return-self.account.init_money)/self.account.init_money
        
        asset_df['year'] = asset_df['date'].apply(lambda x: int(x.split('-')[0]))
        years = set(asset_df['year'].values)
        total_years = len(years)
        
        return round(100.0*avg_rate/total_years,2)
    
    
#   年化收益率=[（投资收益 / 本金）/ 投资天数] × 一年天数 ×100%
    def year_regular_return_rate(self):
        asset_df = self.account.get_asset_df()
        
        last_day_asset = asset_df[asset_df.shape[0]-1:]        
        invest_return = last_day_asset['assets'].values[0]
        
        total_days = asset_df.shape[0]
        
        invest_rate = (invest_return-self.account.init_money)*1.0/self.account.init_money
        year_return_rate_value = 100.0 * 365 * (invest_rate/total_days)
        
        return round(year_return_rate_value,2)
    
    
#   月回报率
    def month_return_rate_array(self):
        asset_df = self.account.asset
        
        asset_df['year'] = asset_df['date'].apply(lambda x: int(x.split('-')[0]))
        asset_df['month'] = asset_df['date'].apply(lambda x: int(x.split('-')[1]))

        years = set(asset_df['year'].values)
        years = list(years)
        years.sort()

        rate_array = []
        
        for each_year in years:
            year_part = asset_df[asset_df['year']==each_year]
            
            months = set(year_part['month'].values)
            months = list(months)
            months.sort()
            
            month_array= []
            for each_month in months:
                month_part = year_part[year_part['month']==each_month]
                
                first_day_of_month = month_part[:1]['assets'].values[0]
                last_day_of_month = month_part[month_part.shape[0]-1:]['assets'].values[0]
                
                month_return_rate = round(100*(last_day_of_month-first_day_of_month)/first_day_of_month,2)

                month_array.append(month_return_rate)
                
            rate_array += month_array
        
        return rate_array
        
    
#   年回报率
    def year_return_rate_array(self):
        asset_df = self.account.asset
        
        asset_df['year'] = asset_df['date'].apply(lambda x: int(x.split('-')[0]))

        years = set(asset_df['year'].values)
        years = list(years)
        years.sort()

        rate_array = []
        
        for each_year in years:
            year_part = asset_df[asset_df['year']==each_year]

            first_day_of_year = year_part[:1]['assets'].values[0]
            last_day_of_year = year_part[year_part.shape[0]-1:]['assets'].values[0]

            year_return_rate = round(100*(last_day_of_year-first_day_of_year)/last_day_of_year,2)
            
            rate_array.append(year_return_rate)
        
        return rate_array
    
    def mean_year_return_rate(self):
        year_returns = self.year_return_rate_array()
        return np.mean(year_returns)
    
    def std_year_return_rate(self):
        year_returns = self.year_return_rate_array()
        return np.std(year_returns,ddof=1)
    
    
    def mean_month_return_rate(self):
        month_returns = self.month_return_rate_array()
        return np.mean(month_returns)
    
    def std_month_return_rate(self):
        month_returns = self.month_return_rate_array()
        return np.std(month_returns,ddof=1)
    
    def mar(self):
        avg_year_return = self.year_avg_return_rate()
        max_drawdown_value = self.get_in_hand_asset_drawdown()
        
        return round(avg_year_return*1.0/max_drawdown_value,2)
    
    def sharp(self,no_risk_return_rate=3):
        cagr_array = self.get_CAGR_array()

        avg_cagr = np.mean(cagr_array)
        std_cagr = np.std(cagr_array,ddof=1)
        return (avg_cagr-no_risk_return_rate)/std_cagr

    
#     盈利月份占比
    def profit_month_rate(self,):
        month_returns = self.month_return_rate_array()
        
        total = len(month_returns)
        profit_count = 0
        for i in range(0,len(month_returns)):
            cur_month = month_returns[i]
            
            if(cur_month>=0):
                profit_count += 1
                
        return round(100.0*profit_count/total,2)
                
    
#     胜率
    def win_rate(self,profits):
        total = len(profits)
        if(total==0):
            return 0
        
        win_count = 0
        for i in range(0,len(profits)):
            cur_profit = profits[i]
            if(cur_profit>=0):
                win_count += 1
                
        return round(100.0*win_count/total,2)
                
    
        
    def continue_loss_times(self,profits):
        if(len(profits)<1):
            return 0
        
        max_loss_count = 0
        
        for i in range(1,len(profits)):
            cur_profit = profits[i]
            
            if(cur_profit<=0):
                loss_count = 0
                j=i;
                while(j>=0 and profits[j]<0):
                    loss_count+=1
                    j -= 1
                if(loss_count>max_loss_count):
                    max_loss_count = loss_count
                
        return max_loss_count
    
    def max_drawdown_per(self, array):
        max_asset = np.maximum.accumulate(array)
        drawndown=max_asset - array
        
        i = np.argmax(drawndown/max_asset) # end of the period
        j = np.argmax(array[:i]) # start of period
        
        return round((1-array[i]/array[j])*100.0,2)
    

    #最大回撤
    def get_float_profit_drawdown(self):
        asset_df = self.account.get_asset_df()
        assets_array = asset_df['assets'].values
        
        return self.max_drawdown_per(assets_array)
    
    
    def get_in_hand_asset_drawdown(self):
        in_hand_asset_array = self.account.get_in_hand_asset()['assets'].values
        
        return self.max_drawdown_per(in_hand_asset_array)
    
    
    def avg_max_drawdown(self,top_per = 5):
        in_hand_asset_df = self.account.get_in_hand_asset()
        
        max_asset = np.maximum.accumulate(in_hand_asset_df['assets'])
        drawndown = max_asset - in_hand_asset_df['assets']
        drawndown_rate = 100.0*drawndown/max_asset
        
        drawndown_array = drawndown_rate.values
        
        drawndown_array.sort()
        drawndown_array = drawndown_array[::-1]
        
        top_n = int(len(drawndown_array)/top_per)
        drawndown_array = drawndown_array[:top_n]

        return np.mean(drawndown_array)
    
        
#   线性回归的斜率
    def linear_model(self,x,y):
        regr = linear_model.LinearRegression()
        regr.fit(x,y)
        
        slope = regr.coef_[0]
        return slope

    
#  这个概念并不复杂。举个例子，你在2005年1月1日最初投资了1万元，而到了2006年1月1日你的资产增长到了1.3万元，
# 到了2007年增长到了1.4万元，而到了2008年1月1日变为1.95万元。
# 根据计算公式，你这笔投资的年复合增长率为：
# 最终资金除以起始资金 ：19,500/10,000 = 1.95
# 开立方（乘以1/年数的次方，这里即乘以1/3次方）
# 1.95^(1/3) =1.2493
# 将结果减去1
# 1.2493 -1=0.2493
# 最后计算获得的CAGR为24.93%，从而意味着你三年的投资回报率为24.93%，即将按年份计算的增长率在时间轴上平坦化。
# 当然，你也看到第一年的增长率则是30%（13000－10000）/10000*100%
#     年复合增长率
    def get_CAGR_array(self):
        asset_df = self.account.asset
        
        asset_df['year'] = asset_df['date'].apply(lambda x: int(x.split('-')[0]))

        years = set(asset_df['year'].values)
        years = list(years)
        years.sort()

        rate_array = []
        
        year_asset_value_array = []
        for each_year in years:
            year_part = asset_df[asset_df['year']==each_year]

            first_day_of_year = year_part[:1]['assets'].values[0]
            
            year_asset_value_array.append(first_day_of_year)
        
        cagr_array = []
        init_asset = year_asset_value_array[0]
        for i in range(1, len(year_asset_value_array)):
            cur_asset = year_asset_value_array[i]
            cagr = round(100.0*(pow(1.0*cur_asset/init_asset,1.0/i)-1),1)
            cagr_array.append(cagr)
            
        return cagr_array
        
    def draw_rar(self,x,y):
        plt.scatter(x,y,color='blue')
        plt.xticks(())
        plt.yticks(())
        plt.show()
    
#   线性回归年回报率
    def get_RAR(self):
        cagr_array = self.get_CAGR_array()
        if(len(cagr_array)<2):
            return None
        
        x = np.array(range(0,len(cagr_array))).reshape(-1, 1)
        y = cagr_array
       
        rar = 100.0*self.linear_model(x,y)
        
        return rar

